**Universidad de Los Andes - Facultad de Economía** <br>
**Machine Learning para Business Intelligence** <br>
**_Paula Rodríguez, Juan S. Moreno, Mateo Dulce_**
# Clase 6: Series de Tiempo

**Lo que cubriremos esta clase:**
1. Introducción: Definiciones y métricas de calidad
2. Mover, suavizar y evaluar series de tiempo
3. Pronósticos:
    - Validación cruzada para series de tiempo
    - Modelos con enfoque econométrico

**Recursos adicionales:**
- Recurso teórico: [_Statistical forecasting: notes on regression and time series analysis, Robert Nau_](https://people.duke.edu/~rnau/411home.htm)
- Recurso práctico (programación): [Time Series Analysis (TSA) in Python - Linear Models to GARCH](http://www.blackarbs.com/blog/time-series-analysis-in-python-linear-models-to-garch/11/1/2016)

## 1. Introducción: Definiciones y métricas de calidad

**_Una serie de tiempo es una serie de puntos de datos indexados (o listados, o graficados) en orden de tiempo._**

Importemos algunas librerías. Primero, necesitaremos la librería [statsmodels] (http://statsmodels.sourceforge.net/stable/), que tiene muchas funciones de modelación estadística, incluidas series de tiempo.

In [ ]:
# vectores y matrices
# tablas y manipulacion de datos
# graficos


# manejo de fechas
# para minimizar funciones
# estadictica y econometria


# alguna funciones utiles


# modo `no molestar`


%matplotlib inline

Como ejemplo, miremos datos reales de juegos móviles. Específicamente, analizaremos los anuncios vistos por hora y el gasto de dinero en el juego por día:

In [ ]:
#cargar datos


In [ ]:
# grafiquemos la serie de tiempo de anuncios


In [ ]:
# grafiquemos la serie de tiempo de gastos diarios en juegos 


### Métricas para pronósticos

Antes de comenzar a pronosticar, veamos cómo medir la calidad de nuestras predicciones.

- [R squared](http://scikit-learn.org/stable/modules/model_evaluation.html#r2-score-the-coefficient-of-determination): coeficiente de determinación (en econometría, esto puede interpretarse como el porcentaje de varianza explicado por el modelo), $(-\infty, 1]$

$R^2 = 1 - \frac{SS_{res}}{SS_{tot}}$ 

```python
sklearn.metrics.r2_score
```
---
- [Mean Absolute Error](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-error): esta es una métrica interpretable porque tiene la misma unidad de medida que la serie inicial, $[0, +\infty)$

$MAE = \frac{\sum\limits_{i=1}^{n} |y_i - \hat{y}_i|}{n}$ 

```python
sklearn.metrics.mean_absolute_error
```
---
- [Median Absolute Error](http://scikit-learn.org/stable/modules/model_evaluation.html#median-absolute-error): nuevamente, una métrica interpretable que es particularmente interesante porque es robusta para los valores atípicos, $[0, +\infty)$

$MedAE = median(|y_1 - \hat{y}_1|, ... , |y_n - \hat{y}_n|)$

```python
sklearn.metrics.median_absolute_error
```
---
- [Mean Squared Error](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error): la métrica más utilizada que otorga una penalización más alta a los errores grandes y viceversa, $[0, +\infty)$

$MSE = \frac{1}{n}\sum\limits_{i=1}^{n} (y_i - \hat{y}_i)^2$

```python
sklearn.metrics.mean_squared_error
```
---
- [Mean Squared Logarithmic Error](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-logarithmic-error): prácticamente, esto es lo mismo que MSE, pero tomamos el logaritmo de la serie. Como resultado, también damos más peso a los pequeños errores. Esto generalmente se usa cuando los datos tienen tendencias exponenciales, $[0, +\infty)$

$MSLE = \frac{1}{n}\sum\limits_{i=1}^{n} (log(1+y_i) - log(1+\hat{y}_i))^2$

```python
sklearn.metrics.mean_squared_log_error
```
---
- Mean Absolute Percentage Error: Esto es lo mismo que MAE, pero se calcula como un porcentaje. $[0, +\infty)$

$MAPE = \frac{100}{n}\sum\limits_{i=1}^{n} \frac{|y_i - \hat{y}_i|}{y_i}$ 

```python
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
```

## 2. Mover, suavizar y evaluar series de tiempo

Comencemos con la idea de que "mañana será lo mismo que hoy". Sin embargo, en lugar de un modelo como $ \hat {y} _ {t} = y_ {t-1} $, asumiremos que el valor futuro de nuestra variable depende del promedio de sus valores anteriores de $ k $. Por lo tanto, utilizaremos el **promedio móvil**.

In [ ]:
# utilizando un promedio movil de 24 horas, cuántos anuncios se verán en 2017-09-22 00:00:00?


Desafortunadamente, no podemos hacer predicciones en el futuro: para obtener el valor para el siguiente paso, necesitamos que se observen los valores anteriores. Pero la media móvil tiene otro caso de uso: **suavizar la serie de tiempo original para identificar tendencias**. </br>

Pandas tiene una implementación disponible con [`DataFrame.rolling (window) .mean ()`] (http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rolling.html). Cuanto más ancha es la ventana, más suave es la tendencia. En el caso de datos muy ruidosos este procedimiento puede ayudar a detectar patrones.

In [ ]:
# usar .rolling para obtener promedio en ventana de 12 horas


In [ ]:
# grafiquemos la serie de tiempo obtenida por promedio movil de 8 horas y la serie de tiempo original


In [ ]:
# definamos una funcion para graficar y poder cambiar la vetana de tiempo
# grafiquemos también los intervalos de confianza para la suavizavión por promedio movil


In [ ]:
# veamos las tendencias diarias


In [ ]:
# graficar promedio movil con ventana de 2 e intervalos de confianza


In [ ]:
# veamos la tendencia semanal del gasto en los juegos


**no capturó la estacionalidad mensual en nuestros datos y marcó casi todos los picos de 30 días como anomalías!!**

- **Promedio ponderado:** $\hat{y}_{t} = \displaystyle\sum^{k}_{n=1} \omega_n y_{t+1-n}$
</br>
</br>
- **Suavizamiento exponencial:** $\hat{y}_{t} = \alpha \cdot y_t + (1-\alpha) \cdot \hat y_{t-1} \quad ; \quad \hat y_{0} = y_{0} $

In [ ]:
# realizar una suavizacion exponencial con alpha = 0.1
#el primer valor es el mismo que el de la serie


In [ ]:
# graficar suavizamiento exponencial con alpha = 0.1


In [ ]:
def suavizamiento_exponencial(series, alpha):
    result = [series[0]]
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result

In [ ]:
def grafSuavExponencial(series, alphas):


In [ ]:
# graficar suav exponencial con alpha de 0.3 y 0.05 para ads


In [ ]:
# graficar suav exponencial con alpha de 0.3 y 0.05 para currency


### Suavizamiento exponencial doble

- $\ell_x = \alpha y_x + (1-\alpha)(\ell_{x-1} + b_{x-1}) \quad \quad \quad$        _Describe el intercepto_ 

- $b_x = \beta(\ell_x - \ell_{x-1}) + (1-\beta)b_{x-1} \quad \quad \quad$ _Describe la tendencia_

- $\hat{y}_{x+1} = \ell_x + b_x \quad \quad \quad \quad \quad \quad \quad \quad \quad$ _Suma de los valores del modelo de la intersección y la tendencia_ 


### Suavizamiento exponencial triple (Holt-Winters)
La idea es agregar un tercer componente: estacionalidad

``statsmodels.tsa.holtwinters``

- $\ell_x = \alpha(y_x - s_{x-L}) + (1-\alpha)(\ell_{x-1} + b_{x-1})$

- $b_x = \beta(\ell_x - \ell_{x-1}) + (1-\beta)b_{x-1}$

- $s_x = \gamma(y_x - \ell_x) + (1-\gamma)s_{x-L}$

- $\hat{y}_{x+m} = \ell_x + mb_x + s_{x-L+1+(m-1)modL}$

## 3. Pronósticos

### 3.1. Validación cruzada para series de tiempo

1. Entrenamos nuestro modelo en un pequeño segmento de la serie temporal desde el principio hasta $t$
2. Hacemos predicciones para los próximos pasos de $t + n$ y calculamos el error
3. Ampliamos nuestra muestra de entrenamiento a $t + n$
4. Hacemos predicciones desde $ t + n $ hasta $ t + 2 * n $
5. Continuamos moviendo nuestro segmento de prueba de la serie de tiempo hasta llegar a la última observación disponible

Como resultado, tenemos tantos pliegues como $ n $ encajarán entre la muestra de entrenamiento inicial y la última observación.

<img src="https://habrastorage.org/files/f5c/7cd/b39/f5c7cdb39ccd4ba68378ca232d20d864.png"/>

## 3.2. Modelos con enfoque econométrico

#### Estacionalidad


Un proceso es **estacionario** si no cambia sus propiedades estadísticas con el tiempo, es decir, su media, varianza y covarianza. (La constancia de la varianza se denomina **homocedasticidad**). La función de covarianza no depende del tiempo; solo debe depender de la distancia entre observaciones. 

**¿Por qué las siguientes series de tiempo son o no estacionarias?**

<img src="https://habrastorage.org/files/20c/9d8/a63/20c9d8a633ec436f91dccd4aedcc6940.png"/>


<img src="https://habrastorage.org/files/b88/eec/a67/b88eeca676d642449cab135273fd5a95.png"/>


<img src="https://habrastorage.org/files/2f6/1ee/cb2/2f61eecb20714352840748b826e38680.png"/>


### Estacionalidad, correlacion y autocorrelación parcial

**Prueba de raíz unitaria de Dickey-Fuller aumentada**

Si se tiene el proceso estocástico en tiempo discreto $\lbrace y_t; t=1,...,\infty \rbrace$ y suponemos que se puede escribir como un proceso autorregresivo de orden $p$:

$$y_t = a_1 y_{t-1} + a_2 y_{t-2} + ...+ a_p y_{t-p} + \epsilon_t$$

Entonces la **ecuación característica** del proceso es:

$$x^p - x^{p-1}a_1 - x^{p-2}a_2 - ...- a_p = 0.$$

Un proceso estocástico lineal tiene una **raíz unitaria** si el valor de la raíz de la ecuación característica del proceso es igual a 1, por lo tanto tal proceso es no estacionario.

La prueba de **Dickey-Fuller** aumentada se puede usar para probar una raíz unitaria en un proceso univariante en presencia de correlación serial.


In [ ]:
# graficar la serie de tiempo de ads e imprimir el p-valor del test de Dickey-Fuller


- **Autocorrelación serial (ACF**): En el rezago k, ésta es la correlación entre los valores de la serie que están separados por k intervalos.

- **Autocorrelación parcial (PACF)**. En el rezago k, ésta es la correlación entre los valores de la serie que están separados por k intervalos, teniendo en cuenta los valores de los intervalos entre ellos.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf


In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf


In [ ]:
def tsplot(y, lags=None, figsize=(12, 7)):

    if not isinstance(y, pd.Series):
        y = pd.Series(y)
        
    fig = plt.figure(figsize=figsize)
    layout = (2, 2)
    ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
    acf_ax = plt.subplot2grid(layout, (1, 0))
    pacf_ax = plt.subplot2grid(layout, (1, 1))

    y.plot(ax=ts_ax)
    p_value = sm.tsa.stattools.adfuller(y)[1]
    ts_ax.set_title('Time Series Analysis Plots\n Dickey-Fuller: p={0:.5f}'.format(p_value))
    smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
    smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
    plt.tight_layout()

In [ ]:
# graficos con lags=60

In [ ]:
# estudiemos la serie de tiempo quitando el componente de periodiciad


In [ ]:
# estuddiemos la serie de tiempo de la primera diferencia


### Modelos ARIMA

Explicaremos este modelo construyendo letra por letra. $ SARIMA (p, d, q) (P, D, Q, s) $, modelo de media móvil de autorregresión estacional (Seasonal Autoregression Moving Average):

- $AR(p)$ - modelo de autorregresión, es decir, regresión de la serie temporal sobre sí misma. La suposición básica es que los valores de la serie actual dependen de sus valores anteriores con algún rezago (o varios rezagos). El rezago máximo en el modelo se conoce como $p$. Para determinar el $p$ inicial, debe mirar el gráfico PACF y encontrar el mayor rezago significativo después del cual **la mayoría** de los otros rezagos se vuelven insignificantes.
- $MA(q)$ - modelo de media móvil. Sin entrar en demasiados detalles, esto modela el error de la serie de tiempo, nuevamente asumiendo que el error actual depende del anterior con algún rezago, lo que se conoce como $q$. El valor inicial se puede encontrar en el gráfico ACF con la misma lógica que antes.

$$AR(p) + MA(q) = ARMA(p, q)$$

- $I(d)$ - orden de integración. Este es simplemente el número de diferencias no estacionales necesarias para que la serie sea estacionaria. En nuestro caso, es solo 1 porque usamos las primeras diferencias.

- $S(s)$ - esto es responsable de la estacionalidad e iguala la duración del período de la temporada de la serie

Con esto tenemos los parámetros: $(P, D, Q)$

In [ ]:
# escogencia de algunos valores iniciales


# creamos una lista con todas las posibles combinaciones de parametros


In [ ]:
def optimizeSARIMA(parameters_list, d, D, s):
    
    results = []
    best_aic = float("inf")

    for param in tqdm_notebook(parameters_list):
        # agregamos un try porque alguna combinaciones de parametros pueden no converger
        try:
            model=sm.tsa.statespace.SARIMAX(ads.Ads, order=(param[0], d, param[1]), 
                                            seasonal_order=(param[2], D, param[3], s)).fit(disp=-1)
        except:
            continue
        aic = model.aic
        # guardamos el mejor modelo, su AIC y sus parametros
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_param = param
        results.append([param, model.aic])

    result_table = pd.DataFrame(results)
    result_table.columns = ['parameters', 'aic']

    result_table = result_table.sort_values(by='aic', ascending=True).reset_index(drop=True)
    
    return result_table

In [ ]:
%%time
tabla_resultados = 

In [ ]:
# seleccionar los parametros que tienen un menos AIC


In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [ ]:
def plotSARIMA(series, model, n_steps):

    # adding model values
    data = series.copy()
    data.columns = ['actual']
    data['arima_model'] = model.fittedvalues
    # making a shift on s+d steps, because these values were unobserved by the model
    # due to the differentiating
    data['arima_model'][:s+d] = np.NaN
    
    # forecasting on n_steps forward 
    forecast = model.predict(start = data.shape[0], end = data.shape[0]+n_steps)
    forecast = data.arima_model.append(forecast)
    # calculate error, again having shifted on s+d steps from the beginning
    error = mean_absolute_percentage_error(data['actual'][s+d:], data['arima_model'][s+d:])

    plt.figure(figsize=(15, 7))
    plt.title("Mean Absolute Percentage Error: {0:.2f}%".format(error))
    plt.plot(forecast, color='r', label="model")
    plt.axvspan(data.index[-1], forecast.index[-1], alpha=0.5, color='lightgrey')
    plt.plot(data.actual, label="actual")
    plt.legend()
    plt.grid(True);

In [ ]:
plotSARIMA(ads, best_model, 50)